# 测试集图像分类预测结果

使用训练好的图像分类模型，预测测试集的所有图像，得到预测结果表格。

## 导入工具包

In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)
import os
fater_path = os.path.abspath('..')
print(fater_path)
os.chdir(fater_path)

device cuda:0
/home/mark/Documents/PythonProject/ShuffleNet-Retrain/ShuffleNetV2+


/home/mark/anaconda3/envs/SemanticTest/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 图像预处理

In [2]:
from torchvision import transforms
import sys
sys.path.append("..")
from COME15KClassDataset import set_data_loader
from torchvision import datasets

# # 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
# train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#                                      ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 载入测试集（和训练代码教程相同） easy

In [3]:
model_path = './models/2023-12-14-17:59_max_epoch_100-afterSOD/'
model_name = 'retrain_COME15K_checkpoint-best-avg-0.544-Medium.pth.tar'

In [4]:
# 数据集文件夹路径
dataset_name = 'test_easy_classes'
dataset_dir = '../data_class_txt/'+ dataset_name + '.txt'
# 类别名称 和 ID索引号 的映射字典
class_names_dic = {0: 'covering', 1: 'device', 2: 'domestic_animal', 3: 'mater', 4: 'person', 5: 'plant',
                       6: 'structure', 7: 'vertebrate'}

test_dataset_loader_easy = set_data_loader(dataset_attr_word="test_easy", batch_size=1, size=512, shuffle=False,
                                           transforms_compose=test_transform, dataset_dir='data/SOD-SemanticDataset')
# 载入测试集
print('测试集图像数量', len(test_dataset_loader_easy))
print('类别个数', len(class_names_dic))
print('各类别名称', list(class_names_dic.values()))
# 获得类别名称
classes = list(class_names_dic.values())
print(classes)

测试集图像数量 4600
类别个数 8
各类别名称 ['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']
['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']


## 载入测试集（和训练代码教程相同）hard

In [5]:
# 数据集文件夹路径
dataset_name = 'test_hard_classes'
dataset_dir = '../data_class_txt/'+ dataset_name + '.txt'
# 类别名称 和 ID索引号 的映射字典
class_names_dic = {0: 'covering', 1: 'device', 2: 'domestic_animal', 3: 'mater', 4: 'person', 5: 'plant',
                       6: 'structure', 7: 'vertebrate'}

test_dataset_loader_easy = set_data_loader(dataset_attr_word="test_hard", batch_size=1, size=512, shuffle=False,
                                           transforms_compose=test_transform, dataset_dir='data/SOD-SemanticDataset')
# 载入测试集
print('测试集图像数量', len(test_dataset_loader_easy))
print('类别个数', len(class_names_dic))
print('各类别名称', list(class_names_dic.values()))
# 获得类别名称
classes = list(class_names_dic.values())
print(classes)

测试集图像数量 3000
类别个数 8
各类别名称 ['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']
['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']


## 导入训练好的模型(导入参数)

In [6]:
# from network import ShuffleNetV2_Plus
# # init model
# architecture = [0, 0, 3, 1, 1, 1, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 2, 1, 3, 2]
# model = ShuffleNetV2_Plus(architecture=architecture, n_class=class_names_dic.__len__(), model_size="Medium")
# weight_path = model_path + model_name
# trained_weight = torch.load(weight_path)
# model.load_state_dict(trained_weight['state_dict'], strict=True)
# model = model.eval().to(device)

## 导入训练好的模型(导入模型 + 参数)

In [7]:
model_and_weight_path = model_path + model_name
model = torch.load(model_and_weight_path)
model = model.eval().to(device)

## 表格A-测试集图像路径及标注

In [8]:
data_path = test_dataset_loader_easy.dataset.data_path

In [9]:
img_paths = test_dataset_loader_easy.dataset.images
img_lables = test_dataset_loader_easy.dataset.labels

In [10]:
df = pd.DataFrame()
df['图像路径'] = [data_path + img_name for img_name in img_paths]
df['标注类别ID'] = img_lables
df['标注类别名称'] = [class_names_dic.get(ID) for ID in img_lables]

In [11]:
df

,图像路径,标注类别ID,标注类别名称
0,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
1,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
2,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
3,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
4,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering
...,...,...,...
2995,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate
2996,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate
2997,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate
2998,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate


## 表格B-测试集每张图像的图像分类预测结果，以及各类别置信度

In [12]:
# 记录 top-n 预测结果
n = 3

In [13]:
df_pred = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    img_path = row['图像路径']
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    
    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = class_names_dic.get(pred_ids[i-1])
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()
        
    df_pred = df_pred._append(pred_dict, ignore_index=True)

3000it [00:50, 59.73it/s]


In [14]:
df_pred

,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,covering-预测置信度,device-预测置信度,domestic_animal-预测置信度,mater-预测置信度,person-预测置信度,plant-预测置信度,structure-预测置信度,vertebrate-预测置信度
0,4,person,0,covering,3,mater,True,0.15070185,0.021558471,0.007850606,0.031099502,0.74086016,0.024640255,0.017243141,0.0060460297
1,3,mater,1,device,2,domestic_animal,False,0.051497,0.14327767,0.09062737,0.4842568,0.08600403,0.015017256,0.06950029,0.05981959
2,4,person,0,covering,3,mater,True,0.2855333,0.023307739,0.034160566,0.05438033,0.53318787,0.029523406,0.01789058,0.022016287
3,4,person,5,plant,0,covering,True,0.11144174,0.07806825,0.008579755,0.065755434,0.49782038,0.1787572,0.039475232,0.020101957
4,3,mater,5,plant,1,device,False,0.061013743,0.15163623,0.040156722,0.29645887,0.080051966,0.20580092,0.08570492,0.07917663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,4,person,2,domestic_animal,7,vertebrate,True,0.1258374,0.038539443,0.18681523,0.056153998,0.38414317,0.0320206,0.047228903,0.12926121
2996,7,vertebrate,2,domestic_animal,4,person,True,0.07538149,0.022477964,0.2160737,0.102507696,0.18334916,0.037898976,0.042219713,0.32009134
2997,7,vertebrate,2,domestic_animal,3,mater,True,0.08253293,0.029319074,0.20741996,0.1760169,0.110707566,0.04276209,0.052742317,0.29849914
2998,3,mater,5,plant,7,vertebrate,True,0.028808666,0.07514538,0.087061,0.27803102,0.039806444,0.26391628,0.043165583,0.18406565


## 拼接AB两张表格

In [15]:
df = pd.concat([df, df_pred], axis=1)

In [16]:
df

,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,covering-预测置信度,device-预测置信度,domestic_animal-预测置信度,mater-预测置信度,person-预测置信度,plant-预测置信度,structure-预测置信度,vertebrate-预测置信度
0,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,4,person,0,covering,3,mater,True,0.15070185,0.021558471,0.007850606,0.031099502,0.74086016,0.024640255,0.017243141,0.0060460297
1,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,3,mater,1,device,2,domestic_animal,False,0.051497,0.14327767,0.09062737,0.4842568,0.08600403,0.015017256,0.06950029,0.05981959
2,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,4,person,0,covering,3,mater,True,0.2855333,0.023307739,0.034160566,0.05438033,0.53318787,0.029523406,0.01789058,0.022016287
3,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,4,person,5,plant,0,covering,True,0.11144174,0.07806825,0.008579755,0.065755434,0.49782038,0.1787572,0.039475232,0.020101957
4,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,0,covering,3,mater,5,plant,1,device,False,0.061013743,0.15163623,0.040156722,0.29645887,0.080051966,0.20580092,0.08570492,0.07917663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,4,person,2,domestic_animal,7,vertebrate,True,0.1258374,0.038539443,0.18681523,0.056153998,0.38414317,0.0320206,0.047228903,0.12926121
2996,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,7,vertebrate,2,domestic_animal,4,person,True,0.07538149,0.022477964,0.2160737,0.102507696,0.18334916,0.037898976,0.042219713,0.32009134
2997,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,7,vertebrate,2,domestic_animal,3,mater,True,0.08253293,0.029319074,0.20741996,0.1760169,0.110707566,0.04276209,0.052742317,0.29849914
2998,data/SOD-SemanticDataset/test/COME15K-Hard/COM...,7,vertebrate,3,mater,5,plant,7,vertebrate,True,0.028808666,0.07514538,0.087061,0.27803102,0.039806444,0.26391628,0.043165583,0.18406565


## 导出完整表格

In [18]:
df.to_csv(model_path + dataset_name +'-测试集预测结果.csv', index=False)